In [1]:
! pip install isodate

In [2]:
import pandas as pd
import numpy as np
import time
from dateutil import parser
import isodate
from googleapiclient.errors import HttpError
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(style="darkgrid", color_codes=True)
from googleapiclient.discovery import build

In [3]:
api_key = ''

channel_ids = ['UC9ctsJZ2aD1nCexfqj342NQ', # BetterBachelor
               'UC53avzpGnh3Kt0tNljlAysw', # alhpa_male_s
               'UCe3lMN6ctO2QVQrqcyyiBzQ', # PrisonPlanetLive
               'UC5sqmi33b7l9kIYa0yASOmQ', # FreshFitMiami
               'UCLIkUEU2t-MR6L6ydFkqAxA', # CorbettUnofficial
               'UCQTAVxA4dNBCoPdHhX9nnoQ', # CoachCoreyWayne
               'UCdHT7KB1gDAXZYpPW71fn0Q', # TheDistributist
               'UCL_f53ZEJxp8TtlOkHwMV9Q', # JordanBPeterson
               'UC1PkRYud11ogYDqgdqd23Zw', # realbenjaminseda
              ]

youtube = build('youtube', 'v3', developerKey=api_key)

In [4]:
def get_channel_stats(youtube, channel_ids):
    """
    Get channel statistics: title, subscriber count, view count, video count, upload playlist
    Params:

    youtube: the build object from googleapiclient.discovery
    channels_ids: list of channel IDs

    Returns:
    Dataframe containing the channel statistics for all channels in the provided list: title, subscriber count, view count, video count, upload playlist

    """
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute()

    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)

    return pd.DataFrame(all_data)

def get_video_ids(youtube, playlist_id):
    """
    Get list of video IDs of all videos in the given playlist
    Params:

    youtube: the build object from googleapiclient.discovery
    playlist_id: playlist ID of the channel

    Returns:
    List of video IDs of all videos in the playlist

    """

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()

    video_ids = []

    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    more_pages = True

    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()

            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])

            next_page_token = response.get('nextPageToken')

    return video_ids

def get_video_details(youtube, video_ids):
    """
    Get video statistics of all videos with given IDs
    Params:

    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs

    Returns:
    Dataframe with statistics of videos, i.e.:
        'channelTitle', 'title', 'description', 'tags', 'publishedAt'
        'viewCount', 'likeCount', 'favoriteCount', 'commentCount'
        'duration', 'definition', 'caption'
    """

    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [5]:
channel_data = get_channel_stats(youtube, channel_ids)

In [6]:
channel_data

,channelName,subscribers,views,totalVideos,playlistId
0,Benjamin Seda,1450000,202690660,1487,UU1PkRYud11ogYDqgdqd23Zw
1,Paul Joseph Watson | Перевод,8,2177,18,UUe3lMN6ctO2QVQrqcyyiBzQ
2,Better Bachelor,475000,231552101,1339,UU9ctsJZ2aD1nCexfqj342NQ
3,Coach Corey Wayne,592000,251139377,6804,UUQTAVxA4dNBCoPdHhX9nnoQ
4,Jordan B Peterson,8630000,946038520,1392,UUL_f53ZEJxp8TtlOkHwMV9Q
5,FreshandFit,1560000,239386718,2269,UU5sqmi33b7l9kIYa0yASOmQ
6,Alhpamales,4,239,5,UU53avzpGnh3Kt0tNljlAysw
7,The Distributist,48500,6870689,404,UUdHT7KB1gDAXZYpPW71fn0Q
8,The Corbett Report (Unofficial),25400,1119602,347,UULIkUEU2t-MR6L6ydFkqAxA


In [7]:
video_df = pd.DataFrame()

for c in channel_data['channelName'].unique():
    print("Getting video information from channel: " + c)
    playlist_id = channel_data.loc[channel_data['channelName']== c, 'playlistId'].iloc[0]
    video_ids = get_video_ids(youtube, playlist_id)

    video_data = get_video_details(youtube, video_ids)

    video_df = pd.concat([video_df, video_data], ignore_index=True)

Getting video information from channel: Benjamin Seda
Getting video information from channel: Paul Joseph Watson | Перевод
Getting video information from channel: Better Bachelor
Getting video information from channel: Coach Corey Wayne
Getting video information from channel: Jordan B Peterson
Getting video information from channel: FreshandFit
Getting video information from channel: Alhpamales
Getting video information from channel: The Distributist
Getting video information from channel: The Corbett Report (Unofficial)


In [8]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,F5eSaabAAmk,Benjamin Seda,How to ACTUALLY Get a Girlfriend in 2025 (Full...,👉🏼 Get 1-3+ dates per week in 30 days (coachin...,"[how to flirt with a girl, dates, how to get a...",2025-03-06T15:27:49Z,5034,254,None,27,PT15M4S,hd,false
1,xJ6b8CV-pQ0,Benjamin Seda,How to Find A 10/10 Girlfriend,👫 My 3 step formula to approach & attract wome...,"[how to flirt with a girl, dates, how to get a...",2025-03-03T15:01:24Z,3346,330,None,22,PT59S,hd,false
2,kPhrei5S88U,Benjamin Seda,The Mistake 99% of Men Make That Keep Them Single,👫 My 3 step formula to approach & attract wome...,"[how to flirt with a girl, dates, how to get a...",2025-03-01T14:45:07Z,2690,222,None,19,PT36S,hd,false
3,4ZnwTwLcAeM,Benjamin Seda,How to Always Get That 2nd Date,👫 My 3 step formula to approach & attract wome...,"[how to flirt with a girl, dates, how to get a...",2025-02-27T14:15:00Z,4060,413,None,9,PT46S,hd,false
4,VW9-SBs6yIg,Benjamin Seda,The Donald Trump Method for Tinder (STEAL THIS),👫 My 3 step formula to approach & attract wome...,"[how to flirt with a girl, dates, how to get a...",2025-02-26T13:45:03Z,6818,316,None,30,PT32S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13670,_X4P6L622-8,The Corbett Report (Unofficial),How Do I Defend Voluntarism? - Questions For C...,I have no affiliation with James Corbett or Th...,None,2022-05-25T13:22:06Z,393,15,None,2,PT28M13S,hd,false
13671,tkmZ4c2AOVY,The Corbett Report (Unofficial),The 5G Dragnet,I have no affiliation with James Corbett or Th...,None,2022-05-24T17:28:15Z,1226,61,None,2,PT25M44S,sd,false
13672,mr7itEUIVew,The Corbett Report (Unofficial),False Flags: The Secret History of Al Qaeda — ...,I have no affiliation with James Corbett or Th...,None,2022-05-24T17:01:10Z,5531,179,None,16,PT1H16M19S,sd,false
13673,ochRNyIDTE8,The Corbett Report (Unofficial),Episode 409 - False Flags: The Secret History ...,I have no affiliation with James Corbett or Th...,None,2022-05-24T17:01:06Z,6132,212,None,16,PT1H59M39S,sd,false


In [14]:
video_data['commentCount'] = pd.to_numeric(video_data['commentCount'])
comment_sum = video_data['commentCount'].sum()
print("There are %d comments in this data frame" % comment_sum)

There are 12672 comments in this data frame


In [15]:
video_df.to_csv('redPillAnalytics.csv', index=False)